In [1]:
import re
import os
import requests
import random
import shutil
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm

random.seed(0)

SOURCE_IN_IMAGES_DIR = "data/dwh/all_images"
TMP_IN_IMAGES_DIR = "data/dwh/good_images"
SELECTED_IN_IMAGES_DIR = "data/dwh/selected_images"

PROCESSED_IMAGES_RIRCI_DIR = "data/dwh/rirci"
PROCESSED_IMAGES_SPLITNET_DIR = "data/dwh/splitnet"
PROCESSED_IMAGES_RAW_DIR = "data/dwh/raw"

In [2]:
df = pd.read_csv('data/dwh/items.csv')
df

,ext_item_id,probability,redflag
0,2008659625,0.999,"Вотермарка <a href=""https://82.img.avito.st/64..."
1,4264413069,0.916,"Вотермарка <a href=""https://48.img.avito.st/64..."
2,7331554917,0.999,"Вотермарка <a href=""https://38.img.avito.st/64..."
3,7331556686,0.999,"Вотермарка <a href=""https://71.img.avito.st/64..."
4,4059684352,0.891,"Вотермарка <a href=""https://55.img.avito.st/64..."
...,...,...,...
995,4409228685,0.999,"Вотермарка <a href=""https://58.img.avito.st/64..."
996,4239258875,0.926,"Вотермарка <a href=""https://22.img.avito.st/64..."
997,3613480973,0.847,"Вотермарка <a href=""https://70.img.avito.st/64..."
998,7247991994,0.999,"Вотермарка <a href=""https://68.img.avito.st/64..."


In [3]:
regex = re.compile(r'href="([^"]*)"')

df['url'] = df['redflag'].apply(
    lambda x: regex.search(x).group(1)
)

In [4]:
df

,ext_item_id,probability,redflag,url
0,2008659625,0.999,"Вотермарка <a href=""https://82.img.avito.st/64...",https://82.img.avito.st/640x480/9196564282.jpg
1,4264413069,0.916,"Вотермарка <a href=""https://48.img.avito.st/64...",https://48.img.avito.st/640x480/40604856448.jpg
2,7331554917,0.999,"Вотермарка <a href=""https://38.img.avito.st/64...",https://38.img.avito.st/640x480/43680099138.jpg
3,7331556686,0.999,"Вотермарка <a href=""https://71.img.avito.st/64...",https://71.img.avito.st/640x480/43567263171.jpg
4,4059684352,0.891,"Вотермарка <a href=""https://55.img.avito.st/64...",https://55.img.avito.st/640x480/28722229955.jpg
...,...,...,...,...
995,4409228685,0.999,"Вотермарка <a href=""https://58.img.avito.st/64...",https://58.img.avito.st/640x480/29594704358.jpg
996,4239258875,0.926,"Вотермарка <a href=""https://22.img.avito.st/64...",https://22.img.avito.st/640x480/39887168522.jpg
997,3613480973,0.847,"Вотермарка <a href=""https://70.img.avito.st/64...",https://70.img.avito.st/640x480/23771810270.jpg
998,7247991994,0.999,"Вотермарка <a href=""https://68.img.avito.st/64...",https://68.img.avito.st/640x480/42591713468.jpg


In [5]:
def download_images(urls, save_folder):
    os.makedirs(save_folder, exist_ok=True)

    for url in tqdm(urls):
        filename = url.split('/')[-1]
        save_path = os.path.join(save_folder, filename)
        
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
    return 0

In [6]:
# download_images(df.url, '/data/dwh_images/')

In [7]:
# from IPython.display import display, clear_output
# import ipywidgets as widgets

# def display_and_filter_images(source_dir, target_dir, max_selections=50):
#     # Ensure directories exist
#     os.makedirs(target_dir, exist_ok=True)
    
#     # Get all image files
#     image_files = [
#         f for f in os.listdir(source_dir)
#         if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))
#     ]
    
#     if not image_files:
#         print("No images found in the source directory.")
#         return
    
#     # Initialize counters
#     total_viewed = 0
#     total_selected = 0
#     total_skipped = 0
    
#     # Create widgets for interaction
#     output = widgets.Output()
#     button_select = widgets.Button(description="Select")
#     button_skip = widgets.Button(description="Skip")
    
#     display(output)
    
#     def show_image(img_file, img_path):
#         nonlocal total_viewed, total_selected, total_skipped
        
#         try:
#             with Image.open(img_path) as img:
#                 width, height = img.size
#                 ratio = max(width, height) / min(width, height)
                
#                 # Skip if aspect ratio > 2:1
#                 if ratio > 2:
#                     total_skipped += 1
#                     with output:
#                         clear_output(wait=True)
#                         plt.figure(figsize=(10, 6))
#                         plt.imshow(img)
#                         plt.axis('off')
#                         plt.title(f"{img_file}\nStatus: AUTO-SKIPPED (bad aspect ratio)", pad=20)
#                         plt.show()
#                         print(f"Viewed: {total_viewed} | Selected: {total_selected} | Skipped: {total_skipped}")
#                     return False
                
#                 total_viewed += 1
#                 with output:
#                     clear_output(wait=True)
#                     plt.figure(figsize=(10, 6))
#                     plt.imshow(img)
#                     plt.axis('off')
#                     plt.title(f"{img_file}\nPlease choose:", pad=20)
#                     plt.show()
#                     print(f"Viewed: {total_viewed} | Selected: {total_selected} | Skipped: {total_skipped}")
#                     display(button_select, button_skip)
#                 return True
                
#         except Exception as e:
#             with output:
#                 clear_output(wait=True)
#                 print(f"Error processing {img_file}: {e}")
#             return False
    
#     def on_select(b):
#         nonlocal total_selected
#         process_selection(select=True)
    
#     def on_skip(b):
#         process_selection(select=False)
    
#     button_select.on_click(on_select)
#     button_skip.on_click(on_skip)
    
#     def process_selection(select):
#         nonlocal total_selected
#         if select:
#             target_path = os.path.join(target_dir, current_img_file)
#             shutil.copy2(current_img_path, target_path)
#             total_selected += 1
#             status = "SELECTED"
#         else:
#             status = "SKIPPED"
        
#         with output:
#             clear_output(wait=True)
#             plt.figure(figsize=(10, 6))
#             plt.imshow(Image.open(current_img_path))
#             plt.axis('off')
#             plt.title(f"{current_img_file}\nStatus: {status}", pad=20)
#             plt.show()
#             print(f"Viewed: {total_viewed} | Selected: {total_selected} | Skipped: {total_skipped}")
        
#         if total_selected < max_selections and image_files:
#             next_image()
    
#     def next_image():
#         nonlocal current_img_file, current_img_path
#         if image_files:
#             current_img_file = image_files.pop(0)
#             current_img_path = os.path.join(source_dir, current_img_file)
#             if not show_image(current_img_file, current_img_path):
#                 next_image()
#         else:
#             with output:
#                 clear_output(wait=True)
#                 print(f"Selection complete!\nViewed: {total_viewed} | Selected: {total_selected} | Skipped: {total_skipped}")
    
#     # Start the process
#     current_img_file = None
#     current_img_path = None
#     next_image()


# display_and_filter_images(SOURCE_IN_IMAGES_DIR, TMP_IN_IMAGES_DIR)

In [8]:
def copy_n_random_files(source, target, n=15):
    if not os.path.isdir(source):
        raise ValueError(f"Source directory does not exist: {source}")

    all_files = [f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f))]
    
    if len(all_files) < n:
        raise ValueError(f"Not enough files in source directory. Requested {n}, found {len(all_files)}")

    selected_files = random.sample(all_files, n)

    if os.path.exists(target):
        shutil.rmtree(target)
    os.makedirs(target)

    for filename in selected_files:
        shutil.copy2(os.path.join(source, filename), target)

    return selected_files

In [9]:
images = copy_n_random_files(TMP_IN_IMAGES_DIR, SELECTED_IN_IMAGES_DIR, n=15)
len(images)

15

### RIRCI inference

In [10]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Available GPUs: {device_count}")
    
    for i in range(device_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("No CUDA devices found.")

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print('Using: ', device)

Available GPUs: 4
GPU 0: NVIDIA A100 80GB PCIe
GPU 1: NVIDIA A100 80GB PCIe
GPU 2: NVIDIA A100 80GB PCIe
GPU 3: NVIDIA A100 80GB PCIe
Using:  cuda:3


In [11]:
from lib.models.rirci import RIRCIModel 
from lib.models.inference_wrapper import InferenceWrapper


def load_model_from_checkpoint(checkpoint_path):
    model = RIRCIModel(simple=True).to(device)
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    model = model.to(device)
    model.eval()

    return InferenceWrapper(model)


model = load_model_from_checkpoint(
    '/app/checkpoints/RIRCIModel-2025-05-10-090853/best_model.pth',
)

/tmp/ipykernel_3765/2975820283.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


In [12]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Resize((480, 640)),
])


def get_rirci_images(model, device):
    if not os.path.isdir(SELECTED_IN_IMAGES_DIR):
        raise ValueError(f"Source directory does not exist: {SELECTED_IN_IMAGES_DIR}")

    if os.path.exists(PROCESSED_IMAGES_RIRCI_DIR):
        shutil.rmtree(PROCESSED_IMAGES_RIRCI_DIR)
    os.makedirs(PROCESSED_IMAGES_RIRCI_DIR)

    processed_files = []
    for filename in os.listdir(SELECTED_IN_IMAGES_DIR):
        src_path = os.path.join(SELECTED_IN_IMAGES_DIR, filename)
        if os.path.isfile(src_path):
            image = preprocess(model.enhance_image(device, src_path, norm=True))
            image.save(os.path.join(PROCESSED_IMAGES_RIRCI_DIR, filename), quality=95)
            processed_files.append(filename)

    return processed_files

processed = get_rirci_images(model, device)
len(processed)

15

In [13]:
# ! git clone https://github.com/vinthony/deep-blind-watermark-removal.git # get code from github
# ! gdown https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc # get pretrained model
# ! mv 27kpng_model_best.pth.tar deep-blind-watermark-removal/

In [14]:
import sys

SPLITNET_DIR = 'deep-blind-watermark-removal'

sys.path.append(SPLITNET_DIR)
sys.path.insert(0, SPLITNET_DIR)

from scripts.utils.imutils import im_to_numpy
import scripts.models as models
import scripts.datasets as datasets

resume_path = f'{SPLITNET_DIR}/27kpng_model_best.pth.tar' # path of pretrained model

In [15]:
model = models.__dict__['vvv4n']().to(device)
model.load_state_dict(torch.load(resume_path)['state_dict'])
model.eval()

model = InferenceWrapper(model)
print('loaded_model')

/opt/conda/lib/python3.11/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/tmp/ipykernel_3765/2929942248.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_s

loaded_model


In [16]:
def get_splitnet_images(model, device):
    if not os.path.isdir(SELECTED_IN_IMAGES_DIR):
        raise ValueError(f"Source directory does not exist: {SELECTED_IN_IMAGES_DIR}")

    if os.path.exists(PROCESSED_IMAGES_SPLITNET_DIR):
        shutil.rmtree(PROCESSED_IMAGES_SPLITNET_DIR)
    os.makedirs(PROCESSED_IMAGES_SPLITNET_DIR)

    processed_files = []
    for filename in os.listdir(SELECTED_IN_IMAGES_DIR):
        src_path = os.path.join(SELECTED_IN_IMAGES_DIR, filename)
        if os.path.isfile(src_path):
            image = preprocess(model.enhance_image(device, src_path, norm=False))
            image.save(os.path.join(PROCESSED_IMAGES_SPLITNET_DIR, filename), quality=95)
            processed_files.append(filename)

    return processed_files

processed = get_splitnet_images(model, device)
len(processed)

15

In [17]:
def open_and_resize(image_path):
    im = Image.open(image_path).convert('RGB')
    return preprocess(im)
    

def get_raw_images(model, device):
    if not os.path.isdir(SELECTED_IN_IMAGES_DIR):
        raise ValueError(f"Source directory does not exist: {SELECTED_IN_IMAGES_DIR}")

    if os.path.exists(PROCESSED_IMAGES_RAW_DIR):
        shutil.rmtree(PROCESSED_IMAGES_RAW_DIR)
    os.makedirs(PROCESSED_IMAGES_RAW_DIR)

    processed_files = []
    for filename in os.listdir(SELECTED_IN_IMAGES_DIR):
        src_path = os.path.join(SELECTED_IN_IMAGES_DIR, filename)
        if os.path.isfile(src_path):
            image = open_and_resize(src_path)
            image.save(os.path.join(PROCESSED_IMAGES_RAW_DIR, filename))
            processed_files.append(filename)

    return processed_files

processed = get_raw_images(model, device)
len(processed)

15

In [18]:
def make_experiment_grid(images):
    n_a, n_b, n_raw = 15, 15, 15

    a_images = images[:n_a]
    b_images = images[:n_b]
    raw_images = images[:n_b]
    
    images = []
    labels = []
   
    for image in a_images:
        images.append(os.path.join(PROCESSED_IMAGES_RIRCI_DIR, image))
        labels.append('rirci')

        
    for image in b_images:
        images.append(os.path.join(PROCESSED_IMAGES_SPLITNET_DIR, image))
        labels.append('splitnet')

    for image in raw_images:
        images.append(os.path.join(PROCESSED_IMAGES_RAW_DIR, image))
        labels.append('raw')
        

    return pd.DataFrame({'image': images, 'label': labels}).sample(frac=1.).reset_index(drop=True)
    

grid = make_experiment_grid(processed)
grid.head()

,image,label
0,data/dwh/raw/24052216339.jpg,raw
1,data/dwh/rirci/25028199228.jpg,rirci
2,data/dwh/raw/25028199228.jpg,raw
3,data/dwh/splitnet/43463297659.jpg,splitnet
4,data/dwh/rirci/37676763127.jpg,rirci


In [19]:
grid.to_csv('data/dwh/experiment_grid.csv', index=False)